In [1]:
%load_ext autoreload

In [10]:
%autoreload
import torch
import matplotlib.pyplot as plt
from gpt.data import ImagenetH5DataModule
from gpt.model import ViT, ClassificationHead, LightningWrapper
from torch.nn import functional as F
from torchvision import transforms

In [8]:
v = LightningWrapper(ViT, image_size=(224, 224), patch_size=(16, 16), image_channels=3, embed_dim=384, mlp_ratio=4, num_heads=8, num_blocks=6, dropout=0.1, lr=1e-3, output_head=ClassificationHead(384, 1000), loss_fn=F.cross_entropy)

/scratch/gpfs/js5013/.conda/envs/foundation/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'output_head' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['output_head'])`.


In [12]:
def collate(lop):
    x, y = zip(*lop)
    x = torch.stack(x)
    return x.view(-1, *x.shape[2:]), torch.tensor(y).unsqueeze(-1).repeat(1, x.shape[1]).view(-1)
    
imagenet = ImagenetH5DataModule(crop_transform=lambda x: x, extra_transforms=[transforms.FiveCrop(224), transforms.Lambda(torch.stack)], batch_size=16, collate_fn=collate)
imagenet.setup()

In [13]:
tl = imagenet.train_dataloader()

In [14]:
x, y = next(iter(tl))

In [25]:
v.loss_fn(v(x), y.long())

tensor(7.0102, grad_fn=<NllLossBackward0>)